# Analysis with full corpus

## Initial analysis

### What other subreddits than r/asianamerican?

### What is the breakdown in number of observations?
- pre-SCOTUS vs post-SCOTUS
- number of rows posted on subreddits other than r/asianamerican
- number of rows for individuals posted both pre and post-SCOTUS

## Broad differences (pre-SCOTUS vs post-SCOTUS)

### Differences in which subreddits users were active on

### Differences in significant collocations

### Differences in sentiment

### Differences in topics

## Individual differences (pre-SCOTUS vs post-SCOTUS)

### Differences in which subreddits users were active on

### Differences in significant collocations

### Differences in sentiment

### Differences in topics